In [ ]:
import glob
import csv
import pandas as pd
from datetime import datetime
import os
import shutil
from tqdm import tqdm
from dateutil import parser
from datetime import datetime
from datetime import timedelta
from collections import Counter

def read_df(path):
    for i in range (0,100):
        try:
            df = pd.read_csv(path, encoding='ISO-8859-1',skiprows=i)
            
            if "date" in " ".join(list(df.columns)).lower() :
                break
        except:
            None
    df = pd.read_csv(path, encoding='ISO-8859-1',skiprows=i, on_bad_lines='skip')
    return df
def process_date(x):
    if "-" in x:
        return x.replace("-","/")
    else:
        return x

def find_dt(dframe):
    for i in dframe.columns:
        if "date" in i.lower() and "time" in i.lower():
            return i
def find_date(dframe):
    for i in dframe.columns:
        if "date" in i.lower() and "time" not in i.lower():
            return i
def find_time(dframe):
    for i in dframe.columns:
        if "date" not in i.lower() and "time" in i.lower():
            return i

def find_temp(dframe):
    for i in dframe.columns:
        if "temp" in i.lower():
            return i
def find_conductivity(dframe):
    for i in dframe.columns:
        if "conductivity" in i.lower() and "specific" not in i.lower():
            return i
            
def find_spec(dframe):
    for i in dframe.columns:
        if "spec" in i.lower() and "cond" in i.lower():
            return i
            
def find_level(dframe):
    for i in dframe.columns:
        if "level" in i.lower():
            return i
            
def find_word(dframe,word):
    for i in dframe.columns:
        if word in i.lower():
            return i

def find_master(listfile, word):
    for i in listfile:
        if word in i.split("/")[-1]:
            return i

def find_datetime_format(a_dataframe_series, mode=None):
    fir = list(set(a_dataframe_series.apply(lambda x: x.split(" ")[0].split("/")[0])))
    sec = list(set(a_dataframe_series.apply(lambda x: x.split(" ")[0].split("/")[1])))
    thi = list(set(a_dataframe_series.apply(lambda x: x.split(" ")[0].split("/")[2])))
    dtset =[fir,sec,thi]
    datetimefdict = {}
    for i in range(3):
        if len(dtset[i][0])==4:
            datetimefdict[i]='%Y'
        else:
            if len(dtset[i])>12:
                datetimefdict[i]='%d'
            elif len(dtset[i])>2:
                datetimefdict[i]='%m'
            else:
                datetimefdict[i]='%Y'
    counter = Counter(datetimefdict)
    date_format_list = [i[-1] for i in sorted(counter.items())]
    if mode == "datetime":
        return "/".join(date_format_list)+ " %H:%M:%S"
    else:
        return "/".join(date_format_list)


allcol = ['DatasetName', 'MonitoringLocationID', 'MonitoringLocationName',
       'MonitoringLocationLatitude', 'MonitoringLocationLongitude',
       'MonitoringLocationHorizontalCoordinateReferenceSystem',
       'MonitoringLocationHorizontalAccuracyMeasure',
       'MonitoringLocationHorizontalAccuracyUnit', 'MonitoringLocationType',
       'ActivityType', 'ActivityMediaName', 'ActivityStartDate',
       'ActivityStartTime', 'ActivityStartTimeZone',
       'SampleCollectionEquipmentName', 'CharacteristicName', 'ResultValue',
       'ResultUnit', 'ResultValueType', 'ResultComment', 'DateTime']

finalcol = ['DatasetName', 'MonitoringLocationID', 'MonitoringLocationName',
       'MonitoringLocationLatitude', 'MonitoringLocationLongitude',
       'MonitoringLocationHorizontalCoordinateReferenceSystem',
       'MonitoringLocationHorizontalAccuracyMeasure',
       'MonitoringLocationHorizontalAccuracyUnit', 'MonitoringLocationType',
       'ActivityType', 'ActivityMediaName', 'ActivityStartDate',
       'ActivityStartTime', 'ActivityStartTimeZone',
       'SampleCollectionEquipmentName', 'CharacteristicName', 'ResultValue',
       'ResultUnit', 'ResultValueType', 'ResultComment']

fixcol = ['DatasetName', 'MonitoringLocationID', 'MonitoringLocationName',
       'MonitoringLocationLatitude', 'MonitoringLocationLongitude',
       'MonitoringLocationHorizontalCoordinateReferenceSystem',
       'MonitoringLocationHorizontalAccuracyMeasure',
       'MonitoringLocationHorizontalAccuracyUnit', 'MonitoringLocationType',
       'ActivityType', 'ActivityMediaName', 'ActivityStartTimeZone',
       'SampleCollectionEquipmentName', 'CharacteristicName',
        'ResultUnit', 'ResultValueType', 'ResultComment']

updatecol = ['ActivityStartDate','ActivityStartTime','ResultValue','DateTime']
def create_master(adf, adfmaster, typeword):
    df_new = pd.DataFrame()
    for i in allcol:
        df_new[i]=[adfmaster[i][0] for _ in range(len(adf))]
    
    adf['ActivityStartDate'] = adf['DateTime'].apply(lambda x:  x.strftime('%Y-%m-%d'))
    adf['ActivityStartTime'] = adf['DateTime'].apply(lambda x:  x.strftime('%H:%M:%S'))
    
    df_new['ActivityStartDate'] = adf['ActivityStartDate']
    df_new['ActivityStartTime'] = adf['ActivityStartTime']
    df_new['ResultValue'] = adf[find_word(adf, typeword)]
    df_new['DateTime'] = adf['DateTime']
    
    return df_new

def drop_error(adf):
    if "ERROR" in adf.columns:
        return adf.drop(columns=['ERROR'])
        
    else:
        return adf

def find_error(adf):
    
    adf['ERROR']= None
    
    if find_dt(adf):
        dt_col= find_dt(adf)
        for idx in range(len(adf[dt_col])):
            try:
                parser.parse(adf[dt_col].loc[idx])
            except:
               adf.loc[idx, "ERROR"] = "DATETIME ERROR"
    else:
        date = find_date(adf)
        time = find_time(adf)
        for idx in range(len(adf)):
            try:
                parser.parse(adf[date].loc[idx])
            except:
                adf.loc[idx, "ERROR"] = "DATETIME ERROR"

            try:
                parser.parse(adf[time].loc[idx])
            except:
                adf.loc[idx, "ERROR"] = "DATETIME ERROR"

    return adf
# path_hobo_solinst = '/home/datpham/cosmo/csv/type/Data files/'
# path_master = '/home/datpham/cosmo/csv/type/'
# listcsv = glob.glob(path_hobo_solinst+"*")
# listmaster = glob.glob(path_master+"*")

# listcsv = [i for i in listcsv if "csv" in i.split("/")[-1]]
# listmaster = [i for i in listmaster if "xlsx" in  i.split("/")[-1]]

def qaqc(list_site_file_link, list_master_file_link, working_path):
    df_con =pd.DataFrame()
    df_tem =pd.DataFrame()
    df_spe =pd.DataFrame()
    df_dep =pd.DataFrame()
    
    
    
    for csvf in tqdm(list_site_file_link):
    
        dfcsv =read_df(csvf)
        #dfcsv = dfcsv.loc[:, (dfcsv != 0).all(axis=0)]   # zeros column not retained
        flag = True
        
        if find_dt(dfcsv):
            #If this file contains date time in one column
            dtc = find_dt(dfcsv)
            dfcsv[dtc] = dfcsv[dtc].apply(lambda x: process_date(x) )

            date_time_format = find_datetime_format(dfcsv[dtc], mode="datetime")
            # yearlength4digit = [1 if len(i)==4 else 0 for i in dfcsv[dtc][0].split(" ")[0].split("/") ]

            # if sum(yearlength4digit)<1:

            #     now = datetime.now()
            #     year2digit = str(now.year)[:2]
            #     dfcsv[dtc] = year2digit + dfcsv[dtc]
            
            try:
                # dfcsv['DateTime'] = dfcsv[dtc].apply(lambda x:  parser.parse(x))
                dfcsv['DateTime'] = dfcsv[dtc].apply(lambda x:  datetime.strptime(x,date_time_format))
                
                remove_num= int(timedelta(hours=3)/(dfcsv['DateTime'].iloc[1] - dfcsv['DateTime'].iloc[0]))
                dfcsv = dfcsv.iloc[:len(dfcsv)-remove_num]
                
            except:
                
                
                # shutil.move(csvf, "/".join(csvf.split("/")[:-1])+ "/outlier/" + csvf.split("/")[-1])
                dfcsv = find_error(dfcsv)
                dfcsv.to_csv(working_path + "/site_files_outlier/" + csvf.split("/")[-1] ,index= False) 
                flag = False
            
        
        else:
            # IF this file have seperate date and time columns
            
            date = find_date(dfcsv)
            dfcsv[date] = dfcsv[date].apply(lambda x: process_date(x) )
            date_format = find_datetime_format( dfcsv[date], mode=None)

            time = find_time(dfcsv)
            try:
                # dfcsv["Date"] = dfcsv['Date'].apply(lambda x:  parser.parse(x))
                dfcsv["Date"] = dfcsv['Date'].apply(lambda x: datetime.strptime(x,date_format))
                
                    
                dfcsv["Time"] = dfcsv['Time'].apply(lambda x:  parser.parse(x))
    
                dfcsv['Date_Time'] = dfcsv["Date"].apply(lambda x:  x.strftime('%Y-%m-%d')) + " " + dfcsv["Time"].apply(lambda x: x.strftime('%H:%M:%S'))
                dfcsv['DateTime'] = dfcsv["Date_Time"].apply(lambda x:  datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))
                
                
            except:
                
                flag = False
    
                dfcsv = find_error(dfcsv)
                dfcsv.to_csv( working_path + "/site_files_outlier/" + csvf.split("/")[-1], index= False)  
                # shutil.move(csvf, "/".join(csvf.split("/")[:-1])+ "/outlier/" + csvf.split("/")[-1])
                
    
    #find_dt find_date find_time find_temp find_conductivity find_spec find_level
    
        if flag:
            con = find_conductivity(dfcsv) 
            if con:
                df_con = pd.concat([df_con, dfcsv[['DateTime']+[con]]])
    
            spec = find_spec(dfcsv) 
            if spec:
                df_spe = pd.concat([df_spe, dfcsv[['DateTime']+[spec]]])
    
            dep = find_level(dfcsv) 
            if dep:
                dfcsv[dep] = dfcsv[dfcsv[dep]<=5][dep]
                
                df_dep = pd.concat([df_dep, dfcsv[['DateTime']+[dep]]])
    
            temp = find_temp(dfcsv) 
            if temp:
                 
                df_tem = pd.concat([df_tem, dfcsv[['DateTime']+[temp]]])
    
    
    try:
        df_con_master = pd.read_csv(find_master(list_master_file_link,"conductivity"), low_memory=False)
        
        df_con_master["Date"] = df_con_master['ActivityStartDate'].apply(lambda x:  datetime.strptime(x,'%m/%d/%Y'))
    
         # 'ActivityStartDate','ActivityStartTime',       
                    
        df_con_master["Time"] = df_con_master['ActivityStartTime'].apply(lambda x:  parser.parse(x))
    
        df_con_master['Date_Time'] = df_con_master["Date"].apply(lambda x:  x.strftime('%Y-%m-%d')) + " " + df_con_master["Time"].apply(lambda x: x.strftime('%H:%M:%S'))
        
        df_con_master['DateTime'] = df_con_master["Date_Time"].apply(lambda x:  datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))
            
    except:
        df_con_master = pd.DataFrame()
    try:
        df_spe_master = pd.read_csv(find_master(list_master_file_link,"spec"), low_memory=False)
        
        df_spe_master["Date"] = df_spe_master['ActivityStartDate'].apply(lambda x:  datetime.strptime(x,'%m/%d/%Y'))
    
         # 'ActivityStartDate','ActivityStartTime',       
                    
        df_spe_master["Time"] = df_spe_master['ActivityStartTime'].apply(lambda x:  parser.parse(x))
    
        df_spe_master['Date_Time'] = df_spe_master["Date"].apply(lambda x:  x.strftime('%Y-%m-%d')) + " " + df_spe_master["Time"].apply(lambda x: x.strftime('%H:%M:%S'))
        
        df_spe_master['DateTime'] = df_spe_master["Date_Time"].apply(lambda x:  datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))

    
    except:
        df_spe_master =pd.DataFrame()
    try:
        df_tem_master = pd.read_csv(find_master(list_master_file_link,"temp"), low_memory=False)

        df_tem_master["Date"] = df_tem_master['ActivityStartDate'].apply(lambda x:  datetime.strptime(x,'%m/%d/%Y'))
    
         # 'ActivityStartDate','ActivityStartTime',       
                    
        df_tem_master["Time"] = df_tem_master['ActivityStartTime'].apply(lambda x:  parser.parse(x))
    
        df_tem_master['Date_Time'] = df_tem_master["Date"].apply(lambda x:  x.strftime('%Y-%m-%d')) + " " + df_tem_master["Time"].apply(lambda x: x.strftime('%H:%M:%S'))
        
        df_tem_master['DateTime'] = df_tem_master["Date_Time"].apply(lambda x:  datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))
        
    except:
        df_tem_master = pd.DataFrame()
    try:
        
        df_dep_master = pd.read_csv(find_master(list_master_file_link,"depth"), low_memory=False)
        
        df_dep_master["Date"] = df_dep_master['ActivityStartDate'].apply(lambda x:  datetime.strptime(x,'%m/%d/%Y'))
    
         # 'ActivityStartDate','ActivityStartTime',       
                    
        df_dep_master["Time"] = df_dep_master['ActivityStartTime'].apply(lambda x:  parser.parse(x))
    
        df_dep_master['Date_Time'] = df_dep_master["Date"].apply(lambda x:  x.strftime('%Y-%m-%d')) + " " + df_dep_master["Time"].apply(lambda x: x.strftime('%H:%M:%S'))
        
        df_dep_master['DateTime'] = df_dep_master["Date_Time"].apply(lambda x:  datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))
    except:
        df_dep_master = pd.DataFrame()
   
    
    if df_con.empty == False and df_con_master.empty == False: 
        df_con = drop_error(df_con)    
        df_con = df_con.sort_values(by='DateTime').reset_index()
        df_con = df_con.drop(columns=['index'])
        df_con = df_con.drop_duplicates(subset='DateTime', keep="first") 
        
        df_con_new = create_master(df_con,df_con_master,"conductivity")
        df_con_new = df_con_new.dropna(axis=0,subset=['ResultValue'])
        df_con_new = df_con_new[df_con_new['DateTime']>df_con_master['DateTime'].iloc[-1]]
        

        if df_con_new.empty == False:
            con_name = find_master(list_master_file_link,"conductivity").split("/")[-1]

            df_con_new[allcol].to_csv(working_path + "/post_processed_files_before_merge_new_data_only/" + con_name ,index= False )
            df_con_new =  pd.concat([df_con_new,df_con_master])
            df_con_new = df_con_new.sort_values(by='DateTime')
    
            df_con_new[allcol].to_csv(working_path + "/post_processed_files_before_merge_appended_old_data/" + con_name ,index= False )  
        else:
        
            con_name = find_master(list_master_file_link,"conductivity").split("/")[-1]
            df_con_master[allcol].to_csv(working_path + "/post_processed_files_before_merge_appended_old_data/" + con_name ,index= False )  
    elif df_con_master.empty == False:
        
        con_name = find_master(list_master_file_link,"conductivity").split("/")[-1]
        df_con_master[allcol].to_csv(working_path + "/post_processed_files_before_merge_appended_old_data/" + con_name ,index= False )
        
        
    if df_spe.empty == False and  df_spe_master.empty == False:
        df_spe = drop_error(df_spe)
        df_spe = df_spe.sort_values(by='DateTime').reset_index()
        df_spe = df_spe.drop(columns=['index'])
        df_spe = df_spe.drop_duplicates(subset='DateTime', keep="first")
        
        df_spe_new = create_master(df_spe,df_spe_master,"spec")
        df_spe_new = df_spe_new.dropna(axis=0,subset=['ResultValue'])
        df_spe_new = df_spe_new[df_spe_new['DateTime']>df_spe_master['DateTime'].iloc[-1]]
        
        

        if df_spe_new.empty == False:
            spe_name = find_master(list_master_file_link,"spec").split("/")[-1]
            df_spe_new[allcol].to_csv(working_path + "/post_processed_files_before_merge_new_data_only/" + spe_name ,index= False )
            
            df_spe_new =  pd.concat([df_spe_new,df_spe_master])
            df_spe_new = df_spe_new.sort_values(by='DateTime')
            
            df_spe_new[allcol].to_csv(working_path + "/post_processed_files_before_merge_appended_old_data/" + spe_name ,index= False )  
        else:
            spe_name = find_master(list_master_file_link,"spec").split("/")[-1]
            
            df_spe_master[allcol].to_csv(working_path + "/post_processed_files_before_merge_appended_old_data/" + spe_name ,index= False )
    elif df_spe_master.empty == False:
        
        spe_name = find_master(list_master_file_link,"spec").split("/")[-1]
            
        df_spe_master[allcol].to_csv(working_path + "/post_processed_files_before_merge_appended_old_data/" + spe_name ,index= False )
    
            
    if df_dep.empty == False and df_dep_master.empty == False :
        df_dep = drop_error(df_dep)
        df_dep = df_dep.sort_values(by='DateTime').reset_index()
        df_dep = df_dep.drop(columns=['index'])
        df_dep = df_dep.drop_duplicates(subset='DateTime', keep="first")   
        
        df_dep_new = create_master(df_dep,df_dep_master,"level")
        df_dep_new = df_dep_new.dropna(axis=0,subset=['ResultValue'])
        df_dep_new = df_dep_new[df_dep_new['DateTime']>df_dep_master['DateTime'].iloc[-1]]
        

        if df_dep_new.empty == False:
            dep_name = find_master(list_master_file_link,"depth").split("/")[-1]
            df_dep_new[allcol].to_csv(working_path + "/post_processed_files_before_merge_new_data_only/" + dep_name ,index= False)
            
            df_dep_new =  pd.concat([df_dep_new,df_dep_master])
            df_dep_new = df_dep_new.sort_values(by='DateTime')

            df_dep_new[allcol].to_csv(working_path + "/post_processed_files_before_merge_appended_old_data/" + dep_name ,index= False)
        else:
            dep_name = find_master(list_master_file_link,"depth").split("/")[-1]
            df_dep_master[allcol].to_csv(working_path + "/post_processed_files_before_merge_appended_old_data/" + dep_name, index= False)

    elif  df_dep_master.empty == False:
        dep_name = find_master(list_master_file_link,"depth").split("/")[-1]
        df_dep_master[allcol].to_csv(working_path + "/post_processed_files_before_merge_appended_old_data/" + dep_name, index= False)
    
        
    if df_tem.empty == False and df_tem_master.empty == False :
        df_tem = drop_error(df_tem)
        df_tem = df_tem.sort_values(by='DateTime').reset_index()
        df_tem = df_tem.drop(columns=['index'])
        df_tem = df_tem.drop_duplicates(subset='DateTime', keep="first")  
        
        df_tem_new = create_master(df_tem,df_tem_master,"temp")

        df_tem_new = df_tem_new.dropna(axis=0,subset=['ResultValue'])
        df_tem_new = df_tem_new[df_tem_new['DateTime']>df_tem_master['DateTime'].iloc[-1]]

        

        if df_tem_new.empty == False:
            tem_name = find_master(list_master_file_link,"temp").split("/")[-1]
            df_tem_new[allcol].to_csv(working_path + "/post_processed_files_before_merge_new_data_only/" + tem_name, index= False)

            df_tem_new =  pd.concat([df_tem_new,df_tem_master])
            df_tem_new = df_tem_new.sort_values(by='DateTime')

            df_tem_new[allcol].to_csv(working_path + "/post_processed_files_before_merge_appended_old_data/" + tem_name, index= False)
        else:
            tem_name = find_master(list_master_file_link,"temp").split("/")[-1]
            df_tem_master[allcol].to_csv(working_path + "/post_processed_files_before_merge_appended_old_data/" + tem_name, index= False)  
    elif  df_tem_master.empty == False:
        tem_name = find_master(list_master_file_link,"temp").split("/")[-1]
        df_tem_master[allcol].to_csv(working_path + "/post_processed_files_before_merge_appended_old_data/" + tem_name, index= False)
        
    

In [ ]:
work_path  = '/lakehouse/default/Files'
list_csv_link = glob.glob(work_path + "/site_files/*")
list_master_link = glob.glob(work_path +"/master_files/*")

print("NUMBER OF SITE FILES: ",len(list_csv_link))
print("NUMBER OF MASTER FILES: ",len(list_master_link))
# list_csv_link = [i for i in list_csv_link if "csv" in i.split("/")[-1]]
# list_master_link = [i for i in list_master_link if "csv" in  i.split("/")[-1]]

site_name_from_csv_list = [i.split("/")[-1].split(" - ")[0] for i in list_csv_link]

site_name_from_master_list = [i.split("/")[-1].split(" - ")[0] for i in list_master_link]

Site_name_from_site_folder_dict ={}
for site in set(site_name_from_csv_list):
    Site_name_from_site_folder_dict[site] = [list_csv_link[idx] for idx,name in enumerate(site_name_from_csv_list) if name == site]

    
Site_name_from_master_folder_dict ={}
for site in set(site_name_from_master_list):
    Site_name_from_master_folder_dict[site] = [list_master_link[idx] for idx,name in enumerate(site_name_from_master_list) if name == site]


for site in tqdm(Site_name_from_site_folder_dict.keys()):
    
    same_site_file_link = Site_name_from_site_folder_dict.get(site)
    
    same_site_master_file_link = Site_name_from_master_folder_dict.get(site)
    if same_site_master_file_link and same_site_file_link:
        print("There are ",len(same_site_file_link), "site files", "for this ",site,". Processing Them")
        qaqc(same_site_file_link, same_site_master_file_link, work_path)
    


path_before_merge = work_path + '/post_processed_files_before_merge_new_data_only/'
listcsv_before_merge = glob.glob(path_before_merge+"*")

df_merged =  pd.DataFrame()
print("PROCESSING SITE FILES COMPLETED, MERGING THEM AND SAVING FINAL FILE...")
for file in listcsv_before_merge:
    df_before_merge = pd.read_csv(file)
    df_merged= pd.concat([df_merged,df_before_merge])
    df_merged = df_merged.sort_values(by=['MonitoringLocationID','DateTime'])
    
    df_merged = df_merged.drop_duplicates()
    df_merged = df_merged.dropna(axis=0,subset=['ResultValue'])
    df_merged[finalcol].to_csv(work_path + "/final_file_ready_to_upload/final_master_file.csv", index= False)
    
print("...SAVING COMPLETED, JOB DONE!")

In [ ]:
# # !pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib
# from googleapiclient.discovery import build
# from google.oauth2 import service_account

# creds = service_account.Credentials.from_service_account_file(
#     "/lakehouse/default/Files/clients_secrets.json",
#     scopes=['https://www.googleapis.com/auth/drive']
# )

# drive_service = build('drive', 'v3', credentials=creds)
# results = (
#         drive_service.files()
#         .list(pageSize=1000)#, fields="nextPageToken, files(id, name)")
#         .execute()
        
#     )
# items = results.get("files", [])